# Splinepy
## The Library
### Splinepy
> Python N-dimensional Bezier, Rational Bezier, BSpline and NURBS library with C++ backend 
It provides all low-level functionality for splines, including
 - import / export for various formats, such as Irit, XML, iges, JSON and MFEM
 - Order Elevation
 - Evaluation
 - Knot Insertion (NURBS, BSplines)
 - Derivatives
 - Compositions of Bezier patches
 - Multiplication and Addition of Bezier-Extracted elements
 - Multipatch fields and boundary identification for pre-processing
 
Splinepy depends on `BSplineLib`, `napf`, `Bezman`. Here, `BSplineLib` is used for NURBS and BSplines, `Bezman` handles all things Bezier. Further `scipy` is regarded as an optional dependency and is sometimes used if available for sparse matrices.

### Other libraries
If you are interested in exploring the C++ backend, we refer to:
- [BSplinelib](https://github.com/tataratat/bsplinelib)
- [BezMan](https://github.com/tataratat/bezman)
- [napf](https://github.com/tataratat/napf)

## Installation
```bash
pip install splinepy[all]
pip install jupyter # for notebook
```

## Getting Started
### Basics


In [ ]:
import splinepy as sp
import gustaf as gus
import numpy as np
import vedo

# comment this line to visualize within the notebook (limited functionalities)
vedo.settings.default_backend = "vtk"

All Spline-Types are constructed using (positional or) the following keyword arguments:
- `degrees` (list)
- `control_points` (list[list] oder `numpy` type)
- `knot_vectors` (list[list])
- `weights`(list of `numpy`-types)

The keywords are used where applicable, i.e.:

| | NURBS | BSpline | Bezier | Rational Bezier |
| -: | :-: | :-: | :-: | :-: |
| `degrees` | X | X | X | X |
| `control_points` | X | X | X | X |
| `knot_vectors` | X | X | - | - |
| `weights` | X | - | - | X |

Let's create some splines:

In [ ]:
# Bezier types
bezier_line = sp.Bezier(degrees=[1], control_points=[[0.0, 0.0], [2, 1]])
bezier_surface = sp.Bezier(
    degrees=[1, 1], control_points=[[0.0, 1.0], [2, 3], [0, 2], [2, 4]]
)

# BSpline
bspline_cube = sp.BSpline(
    degrees=[1, 1, 1],
    control_points=[
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ],
    knot_vectors=[[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 1, 1]],
)

# NURBs
nurbs_line = sp.NURBS(
    degrees=[2],
    knot_vectors=[[0, 0, 0, 0.5, 1, 1, 1]],
    control_points=[[0.0, 1.0], [2, 3], [0, 2], [2, 4]],
    weights=[1, 0.5, 0.8, 1],
)

The transformation into a type with more information is also possible using properties
| FROM \ TO| NURBS | BSpline | Bezier | Rational Bezier |
| -: | :-: | :-: | :-: | :-: |
| NURBS | X | X | X | X |
| BSpline | - | X | X | - |
| Bezier | - | - | X | - |
| Rational Bezier | - | - | X | X |

using

In [ ]:
# Change Type
nurbs_from_bezier_line = bezier_line.nurbs

# Visualize
gus.show(
    [f"{nurbs_from_bezier_line.whatami}", nurbs_from_bezier_line],
    [f"{bezier_line.whatami}", bezier_line]
)

### Construction
Starting from a spline, it can be used as a basis for construction using either revolutions or extrusions.

#### Extrusion

In [ ]:
# Extrude along the vector [1.,0]
extruded_surface = bezier_line.create.extruded(extrusion_vector=[1., 0])

# Show the result
gus.show(
    ["Original", bezier_line],
    ["Extruded along [1., 0] vector", extruded_surface],
)

#### Revolution

In [ ]:
# Revolve along arbitrary axis 
revolved_surface = extruded_surface.create.revolved(
    axis=[1,0,0],
    center=[0,-1,0],
    angle=85,
    degree=True
)
gus.show(
    ["Original", bezier_line],
    ["Extruded along [1., 0] vector", extruded_surface],
    ["Then, revoled", revolved_surface],
)

## Show options
In combination with splines, the `visualize` module has a list of options to modify the look of a given spline. The following list shows the most essential operations:

| Option | Function |
| :----: | :------- |
| `control_points` | Renders the control points in addition to the spline itself (default: `True`) |
| `control_mesh` | Shows the connectivity of control points in the physical domain (default: `True`) |
| `knots`| Plot the knot lines within the physical domain (default: `True`) |
| `c` | Color of the spline | 
| `control_points_c` | Color of control points |
| `control_mesh_c` |  Color of the control mesh |
| `resolutions` | Resolution of the spline as a sampling rate |
| ... | ... |

It is further possible to plot fields and functions and to adjust display options. For more information, please check out the show_options menu.


In [ ]:
# Modify show options
revolved_surface.show_options["c"] = "red"
revolved_surface.show_options["knots"] = True
revolved_surface.show_options["control_mesh"] = False
revolved_surface.show_options["control_point_c"] = "green"
revolved_surface.show_options["control_point_ids"] = True

revolved_surface.show()

### Manipulation
Basic spline refinement strategies are avaible, to modify the parametrization. These include knot insertion and degree elevation, as well as knot removal and degree reduction, where applicable.

For this example, we will start with an arc created with `helpme.create` module.

In [ ]:
curve = sp.helpme.create.arc(radius=4, start_angle=-120, angle=90)
surface = curve.create.revolved(center=curve.control_points[0] + [-5, 1], angle=30).nurbs
surface.show()

#### Degree elevation

In [ ]:
# Prepare copies for comparison
elevated_curve = curve.copy()
elevated_surface = surface.copy()

# Elevate the degree along one specific parametric dimension
elevated_curve.elevate_degrees([0])
# In fact, you can specify multiple degrees at ones
elevated_surface.elevate_degrees([0, 0, 1])

gus.show(
    [f"Original curve. Degrees: {curve.degrees}", curve],
    [f"Degree elevated curve. Degrees: {elevated_curve.degrees}", elevated_curve],
    [f"Original surface. Degrees: {surface.degrees}", surface],
    [f"Degree elevated surface. Degrees: {elevated_surface.degrees}", elevated_surface],
)

#### Knot insertion


In [ ]:
knot_refined_surface = surface.copy()

# insert knots - similar to degree elevation, specify parametric dimension and the location of knots
knot_refined_surface.insert_knots(0, [0.4])
knot_refined_surface.insert_knots(1, [0.3, 0.7])

gus.show(
    ["Original", surface],
    ["After knot insertion", knot_refined_surface],
    ["Parametric space view", knot_refined_surface.create.parametric_view()],
)

#### Knot Removal
This operation is only available for BSplines and NURBS


In [ ]:
knot_removed = knot_refined_surface.copy()
knot_removed.remove_knots(0, [0.4])

gus.show(
    ["Original", knot_refined_surface],
    ["After knot removal", knot_removed],
)

#### Degree reduction
This operation is only available for BSplines and NURBS


In [ ]:
degree_reduced = elevated_surface.copy()
degree_reduced.reduce_degrees([0])

gus.show(
    ["Original", elevated_surface],
    ["After degree reduction", degree_reduced], 
)

## Evaluation
Evaluating splines at specific parametric locations.

In [ ]:
# Evaluate surface
parametric_queries = [
    [0.1, 0.2],
    [0.082, 0.4],
    [0.5, 0.5],
    [0.9, 0.8],
]
evaluated = surface.evaluate(parametric_queries)

# prepare parametric view objects
parametric_view = surface.create.parametric_view()
parametric_view.show_options["alpha"] = 0.7
query_vertices = gus.Vertices(parametric_queries)
query_vertices.show_options["r"] = 5
query_vertices.show_options["c"] = "red"
query_vertices.show_options["vertex_ids"] = True

# physical view objects
surface.show_options["alpha"] = 0.7
surface.show_options["control_points"] = False
physical_vertices = gus.Vertices(evaluated)
physical_vertices.show_options["r"] = 5
physical_vertices.show_options["c"] = "red"
physical_vertices.show_options["vertex_ids"] = True

gus.show(
    ["Parametric View", parametric_view, query_vertices],
    ["Physical View", surface, physical_vertices],
)

## Basis Functions
In order to access basis functions (e.g. for iga-type applications or fitting procedures), use `basis`, `support` or `basis_and_support`:

In [ ]:
queries = np.random.rand(10,1)
basis = nurbs_line.basis(queries)
support = nurbs_line.support(queries)
basis_, support_ = nurbs_line.basis_and_support(queries)
assert np.all(basis == basis_)
assert np.all(support == support_)

You can also use these functions to fill a matrix representing the basis functions at specific positions

In [ ]:
# Create a matrix with basis function values
matrix = np.zeros((queries.shape[0], nurbs_line.control_points.shape[0]))
np.put_along_axis(matrix, support, basis, axis=1)
# Use matrix to compute evaluations
points = matrix @ nurbs_line.control_points
# Here it makes sense to use sparse matrices to save memory and accelerate matrix multiplication
assert np.allclose(points, nurbs_line.evaluate(queries))

Although it is not possible (yet) to plot basis functions directly, you can do so by using a helper function:

In [ ]:
def plot_basis_functions(spline, sample_rate=100, return_fig=False):
    """
    Plot basis functions using vedo plot.
    """
    from vedo.pyplot import plot 

    if spline.para_dim != 1:
        raise ValueError("Only 1D basis functions supported")
        
    n_basis_functions = spline.control_points.shape[0]
    queries = np.linspace(*spline.parametric_bounds, sample_rate)
    basis, supports = spline.basis_and_support(queries)
    basis_function_matrix = np.zeros((sample_rate, n_basis_functions))
    np.put_along_axis(basis_function_matrix, supports, basis, axis=1)
    
    fig = plot(
        queries,
        basis_function_matrix[:,0],
        ylim=(0.,1.01),
        label="B0," + str(spline.degrees[0]),
        title="Basis functions", 
        c=0,
    )
    for i in range(n_basis_functions):
        fig += plot(
            queries,
            basis_function_matrix[:,i],
            ylim=(0.,1.01),
            label="B"+str(i) +"," + str(spline.degrees[0]),
            c=i,
        )
        
    fig.add_legend("top-center", s=0.7)
    if return_fig:
        return fig
    else:
        fig.show()

# nurbs line
basis = plot_basis_functions(nurbs_line, return_fig=True)
gus.show(basis, nurbs_line)

refined_nurbs_line = nurbs_line.copy()
refined_nurbs_line.elevate_degrees([0])
refined_nurbs_line.insert_knots(0, [0.7])
refined_basis = plot_basis_functions(refined_nurbs_line, return_fig=True)

gus.show(
    ["Basis", basis,],
    nurbs_line,
    ["Refined Basis", refined_basis],
    refined_nurbs_line
)

### Derivatives
Splinepy provides several functions to compute derivatives of basis functions and fields, both with respect to parametric, but also with respect to physical coordinates. 

Derivatives with respect to physical coordinates are directly available by their respective member functions.

In [ ]:
# Compute first derivative of the field at u=.5
_ = bezier_line.derivative([[0.5]], [1]) 
# Computes second derivative of the field at u=.5
_ = bezier_line.derivative([[0.5]], [2]) 
# Computes first derivative with respect to second parametric axis
_ = bspline_cube.derivative([[0.2, .4, .3]], [0,1,0]) 
# Mixed derivatives
_ = bspline_cube.derivative([[0.2, .4, .3]], [2,1,3]) # Computes B_{,uuvwww}

If required, the jacobian can also be computed directly, returning a matrix for every query.

In [ ]:
# Compute jacobians
jacs = bspline_cube.jacobian(np.random.rand(100,3))
# Compute determinant of jacobians
dets = np.linalg.det(jacs)

The same holds for basis function derivatives

In [ ]:
# Compute seond order derivative of line basis function at position .5
bfd = bezier_line.basis_derivative([[.5]], [2])
# Compute first order debrivative of basis function with respect to v
bfd = bspline_cube.basis_derivative([[0.2, .4, .3]], [0,1,0])

It is also possible to map the derivatives into physical space, using the provided basis function mapper. The mapper supports gradients, divergences, hessian and laplacians (where applicable).

In [ ]:
# Create a scalar field in 3d parametric space (e.g. a temperature field)
field = sp.Bezier(degrees=[2,2,2],control_points=np.random.rand(27,1))
# Set a geometry
mapper = field.mapper(bspline_cube)
# Compute some hessians and laplacians
queries = np.random.rand(1000,3)
results = mapper.basis_function_derivatives(queries, gradient=True, hessian=True, laplacian=True)
# returns a dictionary with entries "support", "gradient", "hessian", "laplacian"

# You can also compute field values in physical space this way
results = mapper.field_derivatives(queries, gradient=True, divergence=False, hessian=True, laplacian=True, basis_function_values=True)
# returns a dictionary with entries "support", "gradient", "hessian", "laplacian", "basis_function_values"

You can also plot derivatives (or any data) on the spline with a call back function.

In [ ]:
def sample_derivative(spline, resolutions=None, on=None):
    """callback to evaluate derivatives"""
    if resolutions is not None:
        q = gus.create.vertices.raster(
            spline.parametric_bounds, resolutions
        ).vertices
        return spline.derivative(q, [1, 1])
    elif on is not None:
        return spline.derivative(on, [1, 1])

# sampling points
sample_points = gus.create.vertices.raster(
    surface.parametric_bounds, [10, 10]
).vertices

# define spline data
surface.spline_data["derivative"] = sp.SplineDataAdaptor(surface, function=sample_derivative)
surface.show_options.clear()

# mark arrow data to show
surface.show_options["arrow_data"] = "derivative"
surface.show_options["arrow_data_on"] = sample_points

surface.show()


def sample_jacobian_det(spline, resolutions=None, on=None):
    """callback to evaluate jacobian determinants"""
    if resolutions is not None:
        q = gus.create.vertices.raster(
            spline.parametric_bounds, resolutions
        ).vertices
        return np.linalg.det(spline.jacobian(q))
    elif on is not None:
        return np.linalg.det(spline.jacobian(on))

# define spline data
surface.spline_data["jacobian_det"] = sp.SplineDataAdaptor(surface, function=sample_jacobian_det)
surface.show_options.clear()

# mark data name to show - data_name is for scalar data
surface.show_options["lighting"] = "off"
surface.show_options["data_name"] = "jacobian_det"
surface.show_options["arrow_data_on"] = sample_points
surface.show_options["scalarbar"] = True

surface.show()

## Proximity search

Proximity search allows to find the closest point within the spline representation, given physical coordinate. 

In [ ]:
closest_points = bspline_cube.proximities(np.random.rand(100,3))

If no suitable approximation can be identified (for example, if some of the points are outside the physical space) a warning is raised, and the best point is returned, if more information is required, use `return_verbose`-flag to trouble shoot. In some cases, it can also help to increase the number of initial samples, that are used as starting points for the local search.

In [ ]:
# Raises warning
closest_points = bspline_cube.proximities(np.random.rand(100,3))

# Verbose information
verbose_information = bspline_cube.proximities(np.random.rand(100,3), return_verbose=True)

## Bezier Manipulations
Bezier type spline allow for some special operations for spline modification

### Multiplication
Multiplication between two splines is possible, as long as the dimensions of the physical space are compatible. Multiplication of two splines $A(t)$ and $B(t)$ results in a new spline $C$

$ A(t) * B(t) = C(t) \quad \forall t$

In [ ]:
# Scalar Spline as factor
scalar_spline = sp.Bezier(
    degrees=[2],
    control_points=[[1.],[2],[1]]
)
# Vector Spline as factor
vector_spline = sp.Bezier(
    degrees=[1],
    control_points=[[0.,1.],[3.,0]]
)
# Product
product = vector_spline * scalar_spline


# Check results by evaluating at a random point
eval_query = np.random.rand(10,1)
assert np.allclose(
    product.evaluate(eval_query), 
    (vector_spline.evaluate(eval_query)
     * scalar_spline.evaluate(eval_query))
    )

## Addition
As for the Multiplication, Addition of two splines $A(t)$ and $B(t)$ results in a new spline $C$

$ A(t) + B(t) = C(t) \quad \forall t$

As long as the addition of the spline types is defined.

In [ ]:
# Vector Spline as factor
vector_spline = sp.Bezier(
    degrees=[1],
    control_points=[[0.,1.],[3.,0]]
)

# Create a second spline with different orders
second_spline = vector_spline.copy()
second_spline.elevate_degrees(0)

# Sum
sum_spline = vector_spline + second_spline

# Check results by evuating at a random point
eval_query = np.random.rand(10,1)
assert np.allclose(
    sum_spline.evaluate(eval_query), 
    (vector_spline.evaluate(eval_query)
     + vector_spline.evaluate(eval_query))
    )

## Composition
Composition is in the center of microstructure construction. The result of functional composition is a new spline fulfilling 

$ A \circ B = A(B(t))= C(t) \quad \forall t$

Here, the parametric dimension of the outer (or deformation function) must match the physical dimension of the inner function.

In [ ]:
# Inner function (quarter circle)
quarter_circle = sp.RationalBezier(
    degrees=[2],
    control_points=[[1,0],[1,1],[0,1]],
    weights=[1,2**-.5,1]
)

# Outer Function (Rotated Rectangle)
rectangle_surface = sp.Bezier(
    degrees=[1,1],
    control_points=[[.5,0],[1.,.5],[0.,.5],[.5,1.]]
)

# Product
composition = rectangle_surface.compose(quarter_circle)

# Test results
eval_query = np.random.rand(10,1)
assert np.allclose(
    composition.evaluate(eval_query), 
    rectangle_surface.evaluate(
        quarter_circle.evaluate(eval_query)
    )
)

Or into a 3D surface

In [ ]:
# Update Outer Function (Rotated Rectangle)
rectangle_surface = sp.Bezier(
    degrees=[1,1],
    control_points=[[.5,0,0],[1.,.5,.2],[0.,.5,.2],[.5,1.,0]]
)

# Product
composition = rectangle_surface.compose(quarter_circle)

# Test results
eval_query = np.random.rand(10,1)
assert np.allclose(
    composition.evaluate(eval_query), 
    rectangle_surface.evaluate(
        quarter_circle.evaluate(eval_query)
    )
)

## Microstructures
Using the above techniques, it is possible to form microstructures. Here we define a microtile in a unit cube, which is then inserted into the parametric domain of the outer geometry (the so-called deformation function) element-wise.

In [ ]:
microstructure = sp.microstructure.Microstructure()
microstructure.deformation_function = sp.Bezier(
    degrees=[2, 1],
    control_points=[[0, 0], [1, 0], [2, -1], [-1, 1], [1, 1], [3, 2]],
)
microstructure.microtile = [
    sp.Bezier(
        degrees=[3], control_points=[[0, 0.5], [0.5, 1], [0.5, 0], [1, 0.5]]
    ),
    sp.Bezier(
        degrees=[4],
        control_points=[
            [0.5, 0],
            [0.75, 0.5],
            [0.8, 0.8],
            [0.25, 0.5],
            [0.5, 1],
        ],
    ),
]
microstructure.tiling = [7, 5]
microstructure.show(
    control_points=False, title="2D Lattice Microstructure", lighting="off",
)

### Microtiles
splinepy comes with numerous ready-to-use microtiles.

In [ ]:
tile_names = [
    'Armadillo', 
    'CrossTile2D', 
    'CrossTile3D', 
    'Cube3D', 
    'Cubevoid', 
    'DoubleLatticeTile', 
    'Ellipsvoid', 
    'InverseCrossTile3D', 
    'NutTile2D', 
    'NutTile3D', 
    'SnappyTile',
]

micro_tiles = []
for tile_name in tile_names:
    microtile_type = getattr(sp.microstructure.tiles, tile_name)
    micro_tiles.append([microtile_type.__qualname__, microtile_type().create_tile()[0]])


gus.show(
    *micro_tiles,
    resolutions=7,
    control_points=False,
    knots=True,
    alpha=.5,
    lighting="off"
)